# OpenFL - Decentralized Federated Learning on Public Blockchain Systems
### by Anton Wahrstätter, Sajjad Khan & Davor Svetinovic

## CIFAR-10 DATASET GANACHE SIMULATION

### Contents
* [Experiment](#Deploy-Challenger-Contract-and-Start-Experiment)
* [Visualization](#Visualize-Experiment)
* [Individual Transactions](#Transactions)
* [Contracts and Participants](#Contracts-and-Paricipants-(Latex-format))
* [Gas Costs](#Table-with-Gas-Info-(Latex-format))

### Imports
The imports consist of the following:
* PytorchModel - which is the wrapper for the Convolutional Network
* FLManager - the wrapper for the Manager contract
* FLChallenge - the wrapper for the Challenge contract

In [1]:
from pytorch_model import PytorchModel
from smartcontracts import FLManager, FLChallenge

### Select data set

In [2]:
DATASET = "cifar-10"
#DATASET = "mnist"

### RPC Provider

The RPC provides the connection to the respective blockchain, for example Ethereum or the Robsten testnet.

In [3]:
with open("rpc_endpoint.txt", "r") as file:
    RPC_ENDPOINT = file.read()

### Configurations

In [4]:
NUMBER_OF_GOOD_CONTRIBUTORS = 6
NUMBER_OF_BAD_CONTRIBUTORS = 1
NUMBER_OF_FREERIDER_CONTRIBUTORS = 1
NUMBER_OF_INACTIVE_CONTRIBUTORS = 0

REWARD = int(1e18)
MINIMUM_ROUNDS = 25
MIN_BUY_IN = int(1e18)
MAX_BUY_IN = int(1.8e18)
STANDARD_BUY_IN = int(1e18)
EPOCHES = 25
BATCH_SIZE = 128
PUNISHFACTOR = 3
FIRST_ROUND_FEE = 50 # 20% OF MIN DEPOSIT

FORK = False # Fork Chain or communicate directly with RPC

NUMBER_OF_CONTRIBUTERS = NUMBER_OF_GOOD_CONTRIBUTORS      + \
                         NUMBER_OF_BAD_CONTRIBUTORS       + \
                         NUMBER_OF_FREERIDER_CONTRIBUTORS + \
                         NUMBER_OF_INACTIVE_CONTRIBUTORS

In [5]:
# Only for the real-net simulation
# In order to use a non-locally forked blockchain, 
# private keys are required to unlock accounts
if FORK == False:
    from web3 import Web3
    w3 = Web3(Web3.HTTPProvider(RPC_ENDPOINT))
    PRIVKEYS = []
    with open("private_keys.txt", "r") as file:
        for f in file:
            PRIVKEYS.append(f.split(":")[0])

    PRIVKEYS = [w3.eth.account.privateKeyToAccount(i) for i in PRIVKEYS]
else:
    PRIVKEYS = None

### Initialized Deep Learning Model and add Participants

In [6]:
pytorch_model = PytorchModel(DATASET,
                             NUMBER_OF_GOOD_CONTRIBUTORS, 
                             NUMBER_OF_CONTRIBUTERS, 
                             EPOCHES, 
                             BATCH_SIZE, 
                             STANDARD_BUY_IN,
                             MAX_BUY_IN)

for i in range(NUMBER_OF_BAD_CONTRIBUTORS):
    pytorch_model.add_participant("bad",3)

for i in range(NUMBER_OF_FREERIDER_CONTRIBUTORS):
    pytorch_model.add_participant("freerider",1)
    
for i in range(NUMBER_OF_INACTIVE_CONTRIBUTORS):
    pytorch_model.add_participant("inactive",1)

Files already downloaded and verified
Files already downloaded and verified
Data Loaded:
Nr. of images for training: 50,000
Nr. of images for testing:  10,000

Pytorch Model created:

Net_CIFAR(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

Participant added: Good User
Participant added: Good User
Participant added: Good User
Participant added: Good User
Participant added: Good User
Participant added: Good User
Participant added: Bad User
Participant added: Freerider User


### Initialize and Deploy Manager Contract

In [7]:
manager = FLManager(pytorch_model, True).init(NUMBER_OF_GOOD_CONTRIBUTORS, 
                                              NUMBER_OF_BAD_CONTRIBUTORS,
                                              NUMBER_OF_FREERIDER_CONTRIBUTORS,
                                              NUMBER_OF_INACTIVE_CONTRIBUTORS,
                                              MINIMUM_ROUNDS,
                                              RPC_ENDPOINT,
                                              fork=FORK,
                                              accounts=PRIVKEYS)
manager.buildContract()

Connected to Ethereum: True
initiated Ganache-Client @ Block Nr. 0

Total Contributers:       8
Good Contributers:        6 (75%)
Malicious Contributers:   1 (12%)
Freeriding Contributers:  1 (12%)
Inactive Contributers:    0 (0%)
Learning Rounds:          25
-----------------------------------------------------------------------------------
Account initiated @ Address 0x09Ca61C7cb86d8DE76E3E63... with 100,000,000.0 ETH | FAIR USER
Account initiated @ Address 0xe7F50AB14f0fb1E140bbb7f... with 100,000,000.0 ETH | FAIR USER
Account initiated @ Address 0xCe57EAfD18afA405A26BCFa... with 100,000,000.0 ETH | FAIR USER
Account initiated @ Address 0xC0148dFB7E61c11e8422588... with 100,000,000.0 ETH | FAIR USER
Account initiated @ Address 0xa46d9Edaac1d2153168Cdf7... with 100,000,000.0 ETH | FAIR USER
Account initiated @ Address 0xE6d028402A486980cf97375... with 100,000,000.0 ETH | FAIR USER
Account initiated @ Address 0x0be222949e94CdFa24e655A... with 100,000,000.0 ETH | MAL. USER
Account init

### Deploy Challenger Contract and Start Experiment

In [ ]:
configs = manager.deployChallengeContract(MIN_BUY_IN,
                                          MAX_BUY_IN,
                                          REWARD, 
                                          MINIMUM_ROUNDS,
                                          PUNISHFACTOR,
                                          FIRST_ROUND_FEE)

model = FLChallenge(manager, 
                    configs,
                    pytorch_model)


model.simulate(rounds=MINIMUM_ROUNDS)

Starting simulation...
-----------------------------------------------------------------------------------

Model deployed    @ Address 0xAb463Ba18fe9c89D35b886B5d817D16ff4d3F698 | 0x2530...

-----------------------------------------------------------------------------------
Account registered: 0x09Ca61C7cb86d8... | 0x2530... | 1,030,062,952,449,155,072 WEI
Account registered: 0xe7F50AB14f0fb1... | 0x4299... | 1,626,967,045,622,555,136 WEI
Account registered: 0xCe57EAfD18afA4... | 0x0d7f... | 1,242,324,492,475,130,368 WEI
Account registered: 0xC0148dFB7E61c1... | 0xc5f9... | 1,548,517,690,585,905,920 WEI
Account registered: 0xa46d9Edaac1d21... | 0x9b46... | 1,380,338,129,132,500,736 WEI
Account registered: 0xE6d028402A4869... | 0x9566... | 1,035,231,632,641,769,088 WEI
Account registered: 0x0be222949e94Cd... | 0xa646... | 1,637,754,984,685,245,440 WEI
Account registered: 0xE7470ce9261C9f... | 0x357c... | 1,032,228,268,306,087,424 WEI
----------------------------------------------------

### Visualize Experiment 

In [ ]:
model.visualize_simulation()

### Transactions

In [ ]:
print("{:<10} - {:^64} -    Gas Used - {}".format("Function", "Transaction Hash", "Success"))
print("------------------------------------------------------------------------------------------")
for f, txhash in model.txHashes:
    r = model.w3.eth.waitForTransactionReceipt(txhash)
    if r["status"] == 1:
        success = "✅"
    else:
        success = "FAIL"
    
    gas = r["gasUsed"]
    print("{:<10} - {} - {:>9,.0f} -   {}".format(f, txhash, gas, success))

### Contracts and Paricipants (Latex format)

In [ ]:
print("\\renewcommand{\\arraystretch}{1.3}")
print("\\begin{center}")
print("\\begin{tabular}{ c|c }")

print("Contract & Address (Ropsten Testnet) \\\ ")
print("\\hline")
print("Ma-1 & {} \\\ ".format(manager.manager.address))
print("Ch-1 & {} \\\ ".format(model.model.address))
for i, p in enumerate(model.pytorch_model.participants[:-1] + \
                           model.pytorch_model.disqualified + \
                           [model.pytorch_model.participants[-1]]):
    print("P-{}  & {} \\\ ".format(i+1, p.address))

print("\\end{tabular}")
print("\\end{center}")

### Table with Gas Info (Latex format)

In [ ]:
reg = model.gas_register, "register"
fed = model.gas_feedback, "feedback"
clo = model.gas_close, "settle round"
slo = model.gas_slot, "reserve slot"
wei = model.gas_weights, "provide weights**"
dep = manager.gas_deploy, "deployment"
dep = manager.gas_deploy, "deployment"
ext = model.gas_exit, "exit"

tot  = 0
tot2 = 0

print("\\begin{tabular}{ |c|c|c| }\n\hline\nFunction & Gas Amount & Gas Costs*\\\ \n\hline")
for i, f in [reg,slo,wei,fed,clo]:
    print("{} & {:,.0f} & {:.5f} ETH \\\ ".format(f, sum(i)/len(i), sum(i)/len(i) * 20e9 / 1e18 ))
    tot += sum(i)/len(i)
    if i != clo[0]:
            tot2 += sum(i)/len(i)
        
print("\hline\n\hline")
print("complete round & {:,.0f} & {:.5f} \\\ ".format(tot, tot * 20e9 / 1e18))
print("\hline\n\end{tabular}")